# NYC Real Estate Price Prediction Model

In [103]:
import pandas as pd
import numpy as np

In [310]:
df = pd.read_csv('nyc-rolling-sales.csv')
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


We have both categorical and numerical variables

replace column name space with another character

In [311]:
df.columns = df.columns.str.replace(' ', '_')

Drop unwanted columns

In [312]:
df = df.drop(['Unnamed:_0','EASE-MENT','BLOCK','LOT','ADDRESS','APARTMENT_NUMBER','ZIP_CODE'], axis =1)

here is the list of variables

In [313]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         84548 non-null  int64 
 1   NEIGHBORHOOD                    84548 non-null  object
 2   BUILDING_CLASS_CATEGORY         84548 non-null  object
 3   TAX_CLASS_AT_PRESENT            84548 non-null  object
 4   BUILDING_CLASS_AT_PRESENT       84548 non-null  object
 5   RESIDENTIAL_UNITS               84548 non-null  int64 
 6   COMMERCIAL_UNITS                84548 non-null  int64 
 7   TOTAL_UNITS                     84548 non-null  int64 
 8   LAND_SQUARE_FEET                84548 non-null  object
 9   GROSS_SQUARE_FEET               84548 non-null  object
 10  YEAR_BUILT                      84548 non-null  int64 
 11  TAX_CLASS_AT_TIME_OF_SALE       84548 non-null  int64 
 12  BUILDING_CLASS_AT_TIME_OF_SALE  84548 non-null

LAND_SQUARE_FEET, GROSS_SQUARE_FEET and SALE_PRICE has object type but they are actually numerical variables

Investigate for non numeric values

In [315]:
X = df['LAND_SQUARE_FEET'].str.isnumeric()
Y = df['GROSS_SQUARE_FEET'].str.isnumeric()
Z = df['SALE_PRICE'].str.isnumeric()

In [316]:
print(df[~X]['LAND_SQUARE_FEET'].nunique())
print(df[~Y]['GROSS_SQUARE_FEET'].nunique())
print(df[~Z]['SALE_PRICE'].nunique())

1
1
1


In [317]:
print(np.unique(df[~X].iloc[:,8]))
print(np.unique(df[~Y].iloc[:,9]))
print(np.unique(df[~Z].iloc[:,13]))

[' -  ']
[' -  ']
[' -  ']


Because of '-' they are object type variables. Turn these variables to numerical type

In [318]:
numeric = ['LAND_SQUARE_FEET','GROSS_SQUARE_FEET','SALE_PRICE']

df[numeric] = df[numeric].apply(pd.to_numeric, errors = 'coerce')

Split Sale Date to create year column

In [319]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

In [320]:
df['SALE_YEAR'] = pd.DatetimeIndex(df['SALE_DATE']).year

In [321]:
df = df.drop('SALE_DATE', axis =1)

Drop all rows with missing sales prices

In [376]:
df = df.dropna(subset=['SALE_PRICE'])

In [377]:
df_null = df[df['SALE_PRICE'].isin([np.nan,np.inf,-np.inf])]

In [378]:
df_null

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_YEAR


In [379]:
df['SALE_PRICE'].isnull().sum()

0

Creating pipelines to transform numeric and non numeric variables

In [457]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [323]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         84548 non-null  int64  
 1   NEIGHBORHOOD                    84548 non-null  object 
 2   BUILDING_CLASS_CATEGORY         84548 non-null  object 
 3   TAX_CLASS_AT_PRESENT            84548 non-null  object 
 4   BUILDING_CLASS_AT_PRESENT       84548 non-null  object 
 5   RESIDENTIAL_UNITS               84548 non-null  int64  
 6   COMMERCIAL_UNITS                84548 non-null  int64  
 7   TOTAL_UNITS                     84548 non-null  int64  
 8   LAND_SQUARE_FEET                58296 non-null  float64
 9   GROSS_SQUARE_FEET               56936 non-null  float64
 10  YEAR_BUILT                      84548 non-null  int64  
 11  TAX_CLASS_AT_TIME_OF_SALE       84548 non-null  int64  
 12  BUILDING_CLASS_AT_TIME_OF_SALE  

Creating Pipelines for Numeric and Categorical Features 
1. Impute
2. Scale
3. OneHotEncoder


In [420]:
steps_num = [('imputer',SimpleImputer(missing_values = np.nan, strategy='mean')),
             ('scaler', StandardScaler())]
steps_cat = [('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))]
              
numeric_transformer = Pipeline(steps = steps_num)
categorical_transformer = Pipeline(steps = steps_cat)

numeric_features = ['RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT','SALE_YEAR']
categorical_features = ['BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','TAX_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_AT_TIME_OF_SALE']              

In [421]:
preprocessor = ColumnTransformer(transformers = [('num',numeric_transformer,numeric_features),
                                                 ('cat',categorical_transformer,categorical_features)])


Test LinearRegression model

In [422]:
reg = Pipeline(steps = [('preprocessor',preprocessor),
                       ('regressor',LinearRegression())])

In [423]:
y = df.SALE_PRICE
X = df.drop('SALE_PRICE', axis =1)

In [424]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state =42)

In [467]:
clf = reg.fit(X_train,y_train)

In [468]:
y_pred = reg.predict(X_test)
print(reg.predict(X_test))

[1076449.5209352   425105.70028479  731620.75927199 ... 2458868.13655742
 1028636.63034709  467935.57922789]


Calculate loss function

In [469]:
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

13977951.062580628


training performance is not stable??

In [470]:
cv_results = cross_val_score(reg,X_train, y_train, scoring='r2', cv=5)
cv_results

array([-0.78387293,  0.31175609,  0.12227583,  0.42206934,  0.09480307])

In [471]:
cv_results = cross_val_score(reg,X, y, scoring='r2', cv=5)
cv_results

array([-1.49846479e-02, -8.46651572e+00, -2.26346502e+00, -6.15706547e+00,
       -2.04385102e+01])

In [472]:
np.mean(cv_results)

-7.468108206465681

Test KneighborClassifier

In [428]:
clfpipe = Pipeline(steps = [('preprocessor',preprocessor),
                       ('clf',KNeighborsClassifier(n_neighbors =3))])

In [429]:
clfpipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['RESIDENTIAL_UNITS',
                                                   'COMMERCIAL_UNITS',
                                                   'TOTAL_UNITS',
                                                   'LAND_SQUARE_FEET',
                                                   'GROSS_SQUARE_FEET',
                                                   'YEAR_BUILT', 'SALE_YEAR']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BOROUGH', 'NEIGHBORHOOD',
                                                   'BUILDING_CLASS_CATEGORY',
                                                   'TAX_CLASS_AT_PRESENT',
                                                   'BUILDING_CLASS_AT_PRESENT',
                                                   'TAX_CLASS_AT_TIME_OF_SALE',
                                                   'BUILDING_CLASS_AT_TIME_OF_SALE'])])),
                ('clf', KNeighborsClassifier(n_neighbors=3))])

In [430]:
clfpipe.predict(X_test)

array([349830., 823000., 872640., ..., 935000.,      0.,      0.])

In [431]:
clfpipe.score(X_test,y_test)

0.12168405010239558

In [407]:
preprocessor.transformers_[0][1][0]

SimpleImputer()

Used a lot of resources to calculate

In [447]:
#param_grid = {'preprocessor__num__imputer__strategy':['mean','median'],
#              'clf__n_neighbors':np.arange(1,5,2)}

In [450]:
param_grid = {'clf__n_neighbors':np.arange(1,5,2)}

In [451]:
grid_search = GridSearchCV(clfpipe,param_grid,cv=5)

In [452]:
grid_search.fit(X_train,y_train)

/Users/kutayaydin/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['RESIDENTIAL_UNITS',
                                                                          'COMMERCIAL_UNITS',
                                                                          'TOTAL_UNITS',
                                                                          'LAND_SQUARE_FEET',
                                                                          'GROSS_SQUARE_FEET',
                                                                          'YEAR_BUILT',
                                                                          'SALE_YEAR']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['BOROUGH',
                                                                          'NEIGHBORHOOD',
                                                                          'BUILDING_CLASS_CATEGORY',
                                                                          'TAX_CLASS_AT_PRESENT',
                                                                          'BUILDING_CLASS_AT_PRESENT',
                                                                          'TAX_CLASS_AT_TIME_OF_SALE',
                                                                          'BUILDING_CLASS_AT_TIME_OF_SALE'])])),
                                       ('clf',
                                        KNeighborsClassifier(n_neighbors=3))]),
             param_grid={'clf__n_neighbors': array([1, 3])})

In [453]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['RESIDENTIAL_UNITS',
                                                   'COMMERCIAL_UNITS',
                                                   'TOTAL_UNITS',
                                                   'LAND_SQUARE_FEET',
                                                   'GROSS_SQUARE_FEET',
                                                   'YEAR_BUILT', 'SALE_YEAR']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BOROUGH', 'NEIGHBORHOOD',
                                                   'BUILDING_CLASS_CATEGORY',
                                                   'TAX_CLASS_AT_PRESENT',
                                                   'BUILDING_CLASS_AT_PRESENT',
                                                   'TAX_CLASS_AT_TIME_OF_SALE',
                                                   'BUILDING_CLASS_AT_TIME_OF_SALE'])])),
                ('clf', KNeighborsClassifier(n_neighbors=3))])

In [455]:
grid_search.best_score_

0.12520922637272913